# **Chapter 5. [데이터베이스] 쇼핑몰 데이터베이스 만들기**


---
### 📝 **학습 목차**
> 5-1. 프로젝트 개요 <br>
> 5-2. csv 파일 다루기 - csv <br>
> 5-3. 데이터 베이스 연결 - sqlite3 <br>
> 5-4. 데이터 불러오기, 조회하기 - fetchone, fetchall <br>
> 5-5. 데이터 삽입, 수정, 삭제 <br>
> 5-6. 데이터 백업하기 - iterdump <br>
> **5-7. 프로젝트 실습**

## 5-6. 쇼핑몰 데이터 베이스 만들기 🏪

> ### 진행 순서
>  1. 데이터 베이스 생성
>  2. 상품 리스트 테이블 생성
>  3. 주문 목록 추가
>  4. 총 구매 가격 산출

### 5-6-1. 데이터 베이스 생성

In [4]:
import sqlite3

# 쇼핑몰 데이터 베이스
conn = sqlite3.connect("shopping_mall.db")

# 커서 생성
c = conn.cursor()

### 5-6-2. 상품 리스트 테이블 생성

#### csv 파일로 테이블 추가

In [5]:
import pandas as pd

product_list = pd.read_csv('product_list.csv', encoding='euc-kr')
print(product_list)

    ID   상품명        가격
0    1    모자     15000
1    2    코트    200000
2    3   티셔츠     20000
3    4  블라우스     55000
4    5   가디건     45000
5    6   청바지     50000
6    7    구두    150000
7    8    가방    170000
8    9    양말      5000
9    9    양말      5000
10  10    빤쓰   3000000
11  10    빤쓰   3000000
12  10    빤쓰   3000000
13  11   원피스  50000000
14  11   원피스  50000000
15  11   원피스  50000000
16  11   원피스  50000000
17  11   원피스  50000000
18  11   원피스  50000000
19  11   원피스  50000000


In [6]:
# 상품 리스트 테이블 추가
product_list.to_sql('productList', conn, if_exists='append', index = False)

IntegrityError: UNIQUE constraint failed: productList.ID

#### 튜플(리스트)로 테이블 추가

In [4]:
# 테이블 삭제
c.execute("DROP TABLE productList")
conn.commit()

In [5]:
product_list = [['모자',15000],
               ['코트', 200000],
               ['티셔츠',20000],
               ['블라우스',55000],
               ['가디건',45000],
               ['청바지',50000],
               ['구두',150000],
               ['가방',170000],
               ['양말',5000]]

# 테이블 생성
c.execute("CREATE TABLE IF NOT EXISTS productList(ID INTEGER PRIMARY KEY AUTOINCREMENT, 상품명 TEXT, 가격 INTEGER)")

# 레코드 추가
c.executemany("INSERT INTO productList (상품명,가격) values (?,?)", product_list)
conn.commit()

### 5-6-3. 주문 목록 추가

In [6]:
c.execute("CREATE TABLE IF NOT EXISTS orderList(ID INTEGER PRIMARY KEY AUTOINCREMENT, 상품명 TEXT, 개수 INTEGER, 가격 INTEGER, 총합 INTEGER)")
conn.commit()

In [7]:
while True:
    
    ## 상품 목록을 표시
    print("------------------상품목록------------------")
    for row in c.execute('SELECT ID, 상품명, 가격 FROM productList'):
        print('상품번호 :',row[0],', 상품명 :', row[1], ', 가격 :', row[2])
    print("--------------------------------------------")

    
    ## 사용자 상품 선택
    print('')
    num = input("구매하실 상품의 번호를 입력해주세요: ")
    c.execute("SELECT 상품명, 가격 FROM productList WHERE id = ?",(num,))
    result = c.fetchone()
 
 
    ## 상품 번호와 주문 수량을 입력
    print('')
    count = int(input("구매할 수량을 입력해주세요: "))
    total = count * int(result[1])
 

    ## 주문 데이터를 db에 추가
    c.execute("INSERT INTO orderList (상품명, 개수, 가격, 총합) VALUES (?,?,?,?)", (result[0],count,result[1],total))
 
    ## 현재까지 주문 내역을 출력
    print('')
    print("현재까지 구매한 내역 보기")
    print("--------------------주문목록--------------------")
    for row in c.execute('SELECT * FROM orderList'):
        print('상품명 :',row[1],', 주문수량 :', row[2], ', 단가 :', row[3], ', 구매가격 :', row[4])
    print("------------------------------------------------")
 
    ## 사용자 추가 구매 여부 
    print('')
    print("상품을 추가 구매하시겠습니까?\n중단하려면 'x'을 눌러주세요.\n계속 하시려면 엔터키를 눌러주세요. ")
    if(input() == "x"): break
 

------------------상품목록------------------
상품번호 : 1 , 상품명 : 모자 , 가격 : 15000
상품번호 : 2 , 상품명 : 코트 , 가격 : 200000
상품번호 : 3 , 상품명 : 티셔츠 , 가격 : 20000
상품번호 : 4 , 상품명 : 블라우스 , 가격 : 55000
상품번호 : 5 , 상품명 : 가디건 , 가격 : 45000
상품번호 : 6 , 상품명 : 청바지 , 가격 : 50000
상품번호 : 7 , 상품명 : 구두 , 가격 : 150000
상품번호 : 8 , 상품명 : 가방 , 가격 : 170000
상품번호 : 9 , 상품명 : 양말 , 가격 : 5000
--------------------------------------------

구매하실 상품의 번호를 입력해주세요: 3

구매할 수량을 입력해주세요: 1

현재까지 구매한 내역 보기
--------------------주문목록--------------------
상품명 : 티셔츠 , 주문수량 : 1 , 단가 : 20000 , 구매가격 : 20000
------------------------------------------------

상품을 추가 구매하시겠습니까?
중단하려면 'x'을 눌러주세요.
계속 하시려면 엔터키를 눌러주세요. 

------------------상품목록------------------
상품번호 : 1 , 상품명 : 모자 , 가격 : 15000
상품번호 : 2 , 상품명 : 코트 , 가격 : 200000
상품번호 : 3 , 상품명 : 티셔츠 , 가격 : 20000
상품번호 : 4 , 상품명 : 블라우스 , 가격 : 55000
상품번호 : 5 , 상품명 : 가디건 , 가격 : 45000
상품번호 : 6 , 상품명 : 청바지 , 가격 : 50000
상품번호 : 7 , 상품명 : 구두 , 가격 : 150000
상품번호 : 8 , 상품명 : 가방 , 가격 : 170000
상품번호 : 9 , 상품명 : 양말 , 가격 : 5000
-----

### 5-6-4. 총 구매 가격 산출

In [8]:
## 주문 내역에서 총 구매 가격 계산
print("총 구매가격", end='')

for row in c.execute('SELECT sum(총합) FROM orderList'):
    print(' : ',row[0],'원')
print('')

## 주문 내역 초기화
c.execute("DELETE FROM orderList")

## 데이터 베이스 연결 해제
conn.close()

총 구매가격 :  70000 원



### 5-6-5. 전체 실행

In [9]:
import sqlite3

# 쇼핑몰 데이터 베이스
conn = sqlite3.connect("shopping_mall.db", isolation_level=None)
# 커서 생성
c = conn.cursor()


while True:
    
    ## 상품 목록을 표시
    print("------------------상품목록------------------")
    for row in c.execute('SELECT ID, 상품명, 가격 FROM productList'):
        print('상품번호 :',row[0],', 상품명 :', row[1], ', 가격 :', row[2])
    print("--------------------------------------------")

    
    ## 사용자 상품 선택
    print('')
    num = input("구매하실 상품의 번호를 입력해주세요: ")
    c.execute("SELECT 상품명, 가격 FROM productList WHERE id = ?",(num,))
    result = c.fetchone()
 
 
    ## 상품 번호와 주문 수량을 입력
    print('')
    count = int(input("구매할 수량을 입력해주세요: "))
    total = count * int(result[1])  # 주문 수량에 따른 가격
 

    ## 주문 데이터를 db에 추가
    c.execute("INSERT INTO orderList (상품명, 개수, 가격, 총합) VALUES (?,?,?,?)", (result[0],count,result[1],total))
 
    ## 현재까지 주문 내역을 출력
    print('')
    print("현재까지 구매한 내역 보기")
    print("--------------------주문목록--------------------")
    for row in c.execute('SELECT * FROM orderList'):
        print('상품명 :',row[1],', 주문수량 :', row[2], ', 단가 :', row[3], ', 구매가격 :', row[4])
    print("------------------------------------------------")
 
    ## 사용자 추가 구매 여부 
    print('')
    print("상품을 추가 구매하시겠습니까?\n중단하려면 'x'을 눌러주세요.\n계속 하시려면 엔터키를 눌러주세요. ")
    if(input() == "x"): break

## 주문 내역에서 총 구매 가격 계산
print("총 구매가격", end='')

for row in c.execute('SELECT sum(총합) FROM orderList'):
    print(' : ',row[0],'원')
print('')

## 주문 내역 초기화
c.execute("DELETE FROM orderList")

## 데이터 베이스 연결 해제
conn.close()

------------------상품목록------------------
상품번호 : 1 , 상품명 : 모자 , 가격 : 15000
상품번호 : 2 , 상품명 : 코트 , 가격 : 200000
상품번호 : 3 , 상품명 : 티셔츠 , 가격 : 20000
상품번호 : 4 , 상품명 : 블라우스 , 가격 : 55000
상품번호 : 5 , 상품명 : 가디건 , 가격 : 45000
상품번호 : 6 , 상품명 : 청바지 , 가격 : 50000
상품번호 : 7 , 상품명 : 구두 , 가격 : 150000
상품번호 : 8 , 상품명 : 가방 , 가격 : 170000
상품번호 : 9 , 상품명 : 양말 , 가격 : 5000
--------------------------------------------

구매하실 상품의 번호를 입력해주세요: 5

구매할 수량을 입력해주세요: 2

현재까지 구매한 내역 보기
--------------------주문목록--------------------
상품명 : 가디건 , 주문수량 : 2 , 단가 : 45000 , 구매가격 : 90000
------------------------------------------------

상품을 추가 구매하시겠습니까?
중단하려면 'x'을 눌러주세요.
계속 하시려면 엔터키를 눌러주세요. 

------------------상품목록------------------
상품번호 : 1 , 상품명 : 모자 , 가격 : 15000
상품번호 : 2 , 상품명 : 코트 , 가격 : 200000
상품번호 : 3 , 상품명 : 티셔츠 , 가격 : 20000
상품번호 : 4 , 상품명 : 블라우스 , 가격 : 55000
상품번호 : 5 , 상품명 : 가디건 , 가격 : 45000
상품번호 : 6 , 상품명 : 청바지 , 가격 : 50000
상품번호 : 7 , 상품명 : 구두 , 가격 : 150000
상품번호 : 8 , 상품명 : 가방 , 가격 : 170000
상품번호 : 9 , 상품명 : 양말 , 가격 : 5000
-----